# Patch Duration vs. Morphological Calls(63x) per Post Patch Outcomes

##### Date:Update to recent csv by custom reports cell report card 'T301x', 'mIVSCC-MET'

In [1]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline

import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe

from IPython.display import display #Found this to display all columns in a table

#Makes a white backround with gray horizontal gridmarks at y-values
sns.set_style('whitegrid')
sns.set_context('notebook')

#Context: paper
#Context: notebook
#Context: talk
#Context: poster

In [2]:
#Code from Agata
#Opens LIMS, makes a query and then closes LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        #THESE ARE IMPORTANT!!!!!! (Every query needs to be closed when done)
        #This closes query
        cursor.close()             
        conn.close()
    return results


#This last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

##### postpatch_func.py by Rusty from Stash 

In [3]:
def nucleated(x):
    nucleus = x[(x['anno.postPatch_label'] == 'Nucleated') | 
                (((x['anno.postPatch_label'] == 'nucleus_visible') | 
                (x['anno.postPatch_label'] == 'nucleus_present')) & 
                (x['anno.endPipetteR_label'] >= 500))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | 
                (((y['Post patch?'] == 'nucleus_present') | 
                (y['Post patch?'] == 'nucleus_visible')) & 
                (y["anno.endPipetteR_label"] <= 499))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | 
                (((z['Post patch?'] == 'nucleus_absent') | 
                (z['Post patch?'] == 'no_nucleus_visible')) & 
                (z['anno.endPipetteR_label'] >= 500))]
    return outside

def no_seal(w): 
    no = w[(w['anno.postPatch_label'] == 'No-Seal') | 
           (((w['anno.postPatch_label'] == 'nucleus_absent') | 
           (w['anno.postPatch_label'] == 'no_nucleus_visible')) & 
           (w['anno.endPipetteR_label'] <= 499))]
    return no
    
def entire_cell(v):
    entire = v[(v['anno.postPatch_label'] == 'Entire-Cell') | 
               (v['anno.postPatch_label'] == 'entire_cell')]
    return entire

def reclassify(df):
    oo = outside_out(df)
    oo['post patch class'] = 'Outside out'
    nu = nucleated(df)
    nu['post patch class'] = 'Nucleated'
    ns = no_seal(df)
    ns['post patch class'] = 'No seal'
    pn = partial_nucleated(df)
    pn['post patch class'] = 'Partial nucleated'
    ec = entire_cell(df)
    ec['post patch class'] = 'Entire cell'
    return  nu, pn, oo, ns, ec

#out, nuc, noseal, partnuc, ent = reclassify(ps)

def concat_df(a, b, c, d, e):
    frames = (a, b, c, d, e)
    df = pd.concat(frames)
    return df

def postpatch_reclass(df):
    return concat_df(*reclassify(df))

#concat_df(*reclassify(ps)) 
#ps = concat_df(out, nuc, noseal, partnuc, ent)    
#ps

##### Creating lims_df from accessing LIMS

In [4]:
my_query = 'SELECT specimens.name, specimens.patched_cell_container FROM specimens'

lims_df = get_lims_dataframe(my_query)
lims_df.tail()

patched_cell_container                                          name
440500    P8S4_171016_356_A01          Chrna2-Cre_OE25;Ai14-355016.05.02.02
440501    PBS4_180306_502_A01           Pvalb-IRES-Cre;Ai14-378390.04.02.01
440502                   None                Nr5a1-Cre;Ai14-378059.10.06.01
440503                   None                Nr5a1-Cre;Ai14-378059.11.06.01
440504                   None  Nos1-CreERT2;Sst-IRES-FlpO;Ai65-377372.06.02

##### Reads 63x.csv & shiny.csv

In [5]:
morph_df = pd.read_csv("63x.csv")
shiny_df = pd.read_csv("shiny.csv")

##### Creating shiny_df from shiny.csv

In [6]:
shiny_initial = ['anno.sample_id',
                 'anno.Total_time_label',
                 'anno.Ephys_time_label',
                 'anno.Retraction_time_label',
                 'anno.postPatch_label',
                 'anno.endPipetteR_label']
shiny_df = shiny_df[shiny_initial]
shiny_df.head()

anno.sample_id  anno.Total_time_label  anno.Ephys_time_label  \
0  P1S4_160318_069_A01                      0                      0   
1  P1S4_160407_108_A01                      0                      0   
2  P1S4_160407_109_A01                      0                      0   
3  P1S4_160407_112_A01                      0                      0   
4  P1S4_160407_113_A01                      0                      0   

   anno.Retraction_time_label anno.postPatch_label  anno.endPipetteR_label  
0                           0                  O/O                     0.0  
1                           0      nucleated patch                     0.0  
2                           0      partial nucleus                     0.0  
3                           0                  o/o                     0.0  
4                           0                    0                     0.0

##### Merging shiny_df with lims_df to create shiny_lims_df

In [7]:
#Left = shiny 
#Right = LIMS
#Left merge = contains all of shiny but LIMS only displays the matched info with shiny

shiny_lims_df = pd.merge(left = shiny_df, 
                         right = lims_df, 
                         left_on = 'anno.sample_id', 
                         right_on = 'patched_cell_container', 
                         how = 'left')

In [8]:
shiny_lims_df.head()

anno.sample_id  anno.Total_time_label  anno.Ephys_time_label  \
0  P1S4_160318_069_A01                      0                      0   
1  P1S4_160407_108_A01                      0                      0   
2  P1S4_160407_109_A01                      0                      0   
3  P1S4_160407_112_A01                      0                      0   
4  P1S4_160407_113_A01                      0                      0   

   anno.Retraction_time_label anno.postPatch_label  anno.endPipetteR_label  \
0                           0                  O/O                     0.0   
1                           0      nucleated patch                     0.0   
2                           0      partial nucleus                     0.0   
3                           0                  o/o                     0.0   
4                           0                    0                     0.0   

  patched_cell_container                                  name  
0    P1S4_160318_069_A01  Ndnf-IRES2-dgCre;Ai14-234626.02.L.01  
1    P1S4_160407_108_A01  Ndnf-IRES2-dgCre;Ai14-237573.07.R.01  
2    P1S4_160407_109_A01  Ndnf-IRES2-dgCre;Ai14-237573.07.L.01  
3    P1S4_160407_112_A01  Ndnf-IRES2-dgCre;Ai14-237573.07.L.02  
4    P1S4_160407_113_A01  Ndnf-IRES2-dgCre;Ai14-237573.07.L.03

##### Creating morph_df from 63x.csv

In [9]:
morph_initial = ['recording_date',
                'project', 
                'cell_specimen_name', 
                'ims63x_go_no_go',
                'ephys_seal_gohm']
morph_df = morph_df[morph_initial]
morph_df.tail()

recording_date     project  \
6028            NaN  mIVSCC-MET   
6029            NaN  mIVSCC-MET   
6030            NaN  mIVSCC-MET   
6031            NaN  mIVSCC-MET   
6032            NaN  mIVSCC-MET   

                                     cell_specimen_name ims63x_go_no_go  \
6028                  Vip-IRES-Cre;Ai14-363633.04.01.01             NaN   
6029                  Vip-IRES-Cre;Ai14-363633.04.01.02          63x go   
6030  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.03....             NaN   
6031  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.05....             NaN   
6032  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.06....             NaN   

      ephys_seal_gohm  
6028              NaN  
6029              NaN  
6030              NaN  
6031              NaN  
6032              NaN

##### Merging morph_df with shiny_lims_df to create final_df

In [10]:
#Inner merge = contains only matching data of both morph_df and shiny_LIMS_df

final_df = pd.merge(left = morph_df, 
                    right = shiny_lims_df, 
                    left_on = 'cell_specimen_name', 
                    right_on = 'name', 
                    how = 'inner')

In [11]:
final_df.tail(50)

recording_date     project  \
2599            NaN  mIVSCC-MET   
2600            NaN  mIVSCC-MET   
2601            NaN  mIVSCC-MET   
2602            NaN  mIVSCC-MET   
2603            NaN  mIVSCC-MET   
2604            NaN       T301x   
2605            NaN       T301x   
2606            NaN  mIVSCC-MET   
2607            NaN  mIVSCC-MET   
2608            NaN  mIVSCC-MET   
2609            NaN       T301x   
2610            NaN  mIVSCC-MET   
2611            NaN  mIVSCC-MET   
2612            NaN       T301x   
2613            NaN       T301x   
2614            NaN       T301x   
2615            NaN  mIVSCC-MET   
2616            NaN  mIVSCC-MET   
2617            NaN       T301x   
2618            NaN       T301x   
2619            NaN  mIVSCC-MET   
2620            NaN  mIVSCC-MET   
2621            NaN  mIVSCC-MET   
2622            NaN  mIVSCC-MET   
2623            NaN       T301x   
2624            NaN       T301x   
2625            NaN       T301x   
2626            NaN       T301x   
2627            NaN       T301x   
2628            NaN       T301x   
2629            NaN       T301x   
2630            NaN       T301x   
2631            NaN       T301x   
2632            NaN       T301x   
2633            NaN       T301x   
2634            NaN  mIVSCC-MET   
2635            NaN  mIVSCC-MET   
2636            NaN  mIVSCC-MET   
2637            NaN  mIVSCC-MET   
2638            NaN  mIVSCC-MET   
2639            NaN  mIVSCC-MET   
2640            NaN  mIVSCC-MET   
2641            NaN  mIVSCC-MET   
2642            NaN  mIVSCC-MET   
2643            NaN  mIVSCC-MET   
2644            NaN  mIVSCC-MET   
2645            NaN  mIVSCC-MET   
2646            NaN  mIVSCC-MET   
2647            NaN  mIVSCC-MET   
2648            NaN  mIVSCC-MET   

                                     cell_specimen_name ims63x_go_no_go  \
2599                Rbp4-Cre_KL100;Ai14-357010.04.02.03          63x go   
2600                Rbp4-Cre_KL100;Ai14-362155.03.01.01             NaN   
2601                Rbp4-Cre_KL100;Ai14-362155.03.02.01             NaN   
2602                Rbp4-Cre_KL100;Ai14-362155.05.02.01             NaN   
2603                Rbp4-Cre_KL100;Ai14-366313.03.01.01          63x go   
2604                Rorb-IRES2-Cre;Ai14-318659.04.01.02             NaN   
2605                Rorb-IRES2-Cre;Ai14-318659.04.02.02             NaN   
2606                Rorb-IRES2-Cre;Ai14-355888.04.01.01             NaN   
2607                Rorb-IRES2-Cre;Ai14-355888.04.02.01             NaN   
2608                Rorb-IRES2-Cre;Ai14-355888.06.02.01             NaN   
2609                Scnn1a-Tg2-Cre;Ai14-346637.04.01.01             NaN   
2610              Scnn1a-Tg2-Cre;Ai14-358945.0.03.01.01             NaN   
2611              Scnn1a-Tg2-Cre;Ai14-358945.0.06.01.01          63x go   
2612              Slc32a1-IRES-Cre;Ai14-348998.03.02.01       63x no go   
2613              Slc32a1-IRES-Cre;Ai14-348998.04.02.01       63x no go   
2614              Slc32a1-IRES-Cre;Ai14-348998.06.02.01             NaN   
2615              Slc32a1-IRES-Cre;Ai14-364558.04.01.01             NaN   
2616              Slc32a1-IRES-Cre;Ai14-364559.05.02.01          63x go   
2617                  Sst-IRES-Cre;Ai14-346909.03.02.01             NaN   
2618                  Sst-IRES-Cre;Ai14-346909.03.02.02             NaN   
2619                  Sst-IRES-Cre;Ai14-357932.04.01.01             NaN   
2620                 Tlx3-Cre_PL56;Ai14-353877.03.01.01             NaN   
2621                 Tlx3-Cre_PL56;Ai14-353877.03.02.01             NaN   
2622                 Tlx3-Cre_PL56;Ai14-353877.06.01.01             NaN   
2623                  Vip-IRES-Cre;Ai14-347132.02.02.01             NaN   
2624                  Vip-IRES-Cre;Ai14-347132.02.02.02             NaN   
2625                  Vip-IRES-Cre;Ai14-347132.02.02.03             NaN   
2626                  Vip-IRES-Cre;Ai14-347132.03.02.01             NaN   
2627                  Vip-IRES-Cre;Ai14-3471

##### Creating Date column using patched_cell_container column

In [12]:
final_df['Date'] = final_df['patched_cell_container'].str[5:11]

In [13]:
final_df.tail()

recording_date     project  \
2644            NaN  mIVSCC-MET   
2645            NaN  mIVSCC-MET   
2646            NaN  mIVSCC-MET   
2647            NaN  mIVSCC-MET   
2648            NaN  mIVSCC-MET   

                                     cell_specimen_name ims63x_go_no_go  \
2644                  Vip-IRES-Cre;Ai14-363633.04.01.01             NaN   
2645                  Vip-IRES-Cre;Ai14-363633.04.01.02          63x go   
2646  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.03....             NaN   
2647  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.05....             NaN   
2648  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.06....             NaN   

      ephys_seal_gohm       anno.sample_id  anno.Total_time_label  \
2644              NaN  P6S4_171219_251_A01                    447   
2645              NaN  P6S4_171219_252_A01                    397   
2646              NaN  PBS4_171013_501_A01                    319   
2647              NaN  P8S4_171013_351_A01                 -39114   
2648              NaN  PAS4_171013_451_A01                    443   

      anno.Ephys_time_label  anno.Retraction_time_label anno.postPatch_label  \
2644                     98                         151      nucleus_present   
2645                     89                         126      nucleus_present   
2646                      2                           0       nucleus_absent   
2647                    300                      -39418       nucleus_absent   
2648                    300                          29       nucleus_absent   

      anno.endPipetteR_label patched_cell_container  \
2644                  1500.0    P6S4_171219_251_A01   
2645                  2500.0    P6S4_171219_252_A01   
2646                     0.0    PBS4_171013_501_A01   
2647                     8.0    P8S4_171013_351_A01   
2648                    15.0    PAS4_171013_451_A01   

                                                   name    Date  
2644                  Vip-IRES-Cre;Ai14-363633.04.01.01  171219  
2645                  Vip-IRES-Cre;Ai14-363633.04.01.02  171219  
2646  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.03....  171013  
2647  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.05....  171013  
2648  Vipr2-IRES2-Cre;Pvalb-T2A-FlpO;Ai65-349552.06....  171013

##### Indexing Date column then sorting by date index

In [14]:
final_df = final_df.set_index('Date')
final_df = final_df.sort_index()

In [15]:
final_df.head()

recording_date project                     cell_specimen_name  \
Date                                                                     
160510  5/10/2016 10:48   T301x  Rorb-IRES2-Cre-D;Ai14-245133.07.01.01   
160511  5/11/2016 11:38   T301x    Scnn1a-Tg2-Cre;Ai14-245345.02.02.01   
160511  5/11/2016 13:07   T301x    Scnn1a-Tg2-Cre;Ai14-245345.07.01.01   
160511  5/11/2016 13:48   T301x    Scnn1a-Tg2-Cre;Ai14-245345.07.01.02   
160512  5/12/2016 14:24   T301x  Rorb-IRES2-Cre-D;Ai14-245132.07.02.04   

       ims63x_go_no_go  ephys_seal_gohm       anno.sample_id  \
Date                                                           
160510             NaN         1.158136  P1S4_160510_455_A01   
160511             NaN         1.045532  P1S4_160511_471_A01   
160511             NaN         4.639288  P1S4_160511_472_A01   
160511             NaN         1.187988  P1S4_160511_473_A01   
160512             NaN         3.153809  P1S4_160512_485_A01   

        anno.Total_time_label  anno.Ephys_time_label  \
Date                                                   
160510                      0                      0   
160511                      0                      0   
160511                      0                      0   
160511                      0                      0   
160512                      0                      0   

        anno.Retraction_time_label anno.postPatch_label  \
Date                                                      
160510                           0                    0   
160511                           0                    0   
160511                           0      Partial nucleus   
160511                           0                    0   
160512                           0                  o/o   

        anno.endPipetteR_label patched_cell_container  \
Date                                                    
160510                     0.0    P1S4_160510_455_A01   
160511                     0.0    P1S4_160511_471_A01   
160511                     0.0    P1S4_160511_472_A01   
160511                     0.0    P1S4_160511_473_A01   
160512                     0.0    P1S4_160512_485_A01   

                                         name  
Date                                           
160510  Rorb-IRES2-Cre-D;Ai14-245133.07.01.01  
160511    Scnn1a-Tg2-Cre;Ai14-245345.02.02.01  
160511    Scnn1a-Tg2-Cre;Ai14-245345.07.01.01  
160511    Scnn1a-Tg2-Cre;Ai14-245345.07.01.02  
160512  Rorb-IRES2-Cre-D;Ai14-245132.07.02.04

In [16]:
final_df.tail()

recording_date     project                   cell_specimen_name  \
Date                                                                       
180213  2/13/2018 19:05  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.03   
180213  2/13/2018 18:09  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.01.01   
180213  2/13/2018 18:15  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.01   
180213  2/13/2018 18:41  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.02   
180213  2/13/2018 18:45  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.04.01.01   

       ims63x_go_no_go  ephys_seal_gohm       anno.sample_id  \
Date                                                           
180213             NaN         2.400271  PBS4_180213_503_A01   
180213             NaN         2.483585  P9S4_180213_401_A01   
180213             NaN         1.208281  PBS4_180213_501_A01   
180213             NaN         2.977548  PBS4_180213_502_A01   
180213             NaN         4.834966  P1S4_180213_001_A01   

        anno.Total_time_label  anno.Ephys_time_label  \
Date                                                   
180213                    672                     45   
180213                    838                     43   
180213                    921                     46   
180213                    948                     37   
180213                    497                     48   

        anno.Retraction_time_label anno.postPatch_label  \
Date                                                      
180213                           0       nucleus_absent   
180213                         231       nucleus_absent   
180213                           0      nucleus_present   
180213                           0       nucleus_absent   
180213                          78       nucleus_absent   

        anno.endPipetteR_label patched_cell_container  \
Date                                                    
180213                     0.0    PBS4_180213_503_A01   
180213                   900.0    P9S4_180213_401_A01   
180213                     0.0    PBS4_180213_501_A01   
180213                  1000.0    PBS4_180213_502_A01   
180213                  1500.0    P1S4_180213_001_A01   

                                       name  
Date                                         
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.03  
180213  Pvalb-IRES-Cre;Ai14-375845.03.01.01  
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.01  
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.02  
180213  Pvalb-IRES-Cre;Ai14-375845.04.01.01

##### final_df start at production date to present date

In [17]:
final_df = final_df['170901':]

In [18]:
final_df.head()

recording_date project                    cell_specimen_name  \
Date                                                                   
170901  9/1/2017 17:46   T301x  Htr3a-Cre_NO152;Ai14-343221.05.01.01   
170901  9/1/2017 17:36   T301x  Htr3a-Cre_NO152;Ai14-343221.05.02.01   
170901  9/1/2017 20:09   T301x   Scnn1a-Tg3-Cre;Ai14-346622.05.02.01   
170901  9/1/2017 20:47   T301x   Scnn1a-Tg3-Cre;Ai14-346622.04.02.01   
170901  9/1/2017 20:31   T301x   Scnn1a-Tg3-Cre;Ai14-346622.05.02.02   

       ims63x_go_no_go  ephys_seal_gohm       anno.sample_id  \
Date                                                           
170901          63x go         2.897146  P2S4_170901_051_A01   
170901       63x no go         2.461183  P8S4_170901_351_A01   
170901       63x no go         3.079292  P8S4_170901_353_A01   
170901          63x go         1.452649  P2S4_170901_054_A01   
170901       63x no go         2.386576  P8S4_170901_354_A01   

        anno.Total_time_label  anno.Ephys_time_label  \
Date                                                   
170901                      0                      0   
170901                      0                      0   
170901                      0                      0   
170901                      0                      0   
170901                      0                      0   

        anno.Retraction_time_label anno.postPatch_label  \
Date                                                      
170901                           0            Nucleated   
170901                           0              No-Seal   
170901                           0              No-Seal   
170901                           0      Partial-Nucleus   
170901                           0              No-Seal   

        anno.endPipetteR_label patched_cell_container  \
Date                                                    
170901                     0.0    P2S4_170901_051_A01   
170901                     0.0    P8S4_170901_351_A01   
170901                     0.0    P8S4_170901_353_A01   
170901                     0.0    P2S4_170901_054_A01   
170901                     0.0    P8S4_170901_354_A01   

                                        name  
Date                                          
170901  Htr3a-Cre_NO152;Ai14-343221.05.01.01  
170901  Htr3a-Cre_NO152;Ai14-343221.05.02.01  
170901   Scnn1a-Tg3-Cre;Ai14-346622.05.02.01  
170901   Scnn1a-Tg3-Cre;Ai14-346622.04.02.01  
170901   Scnn1a-Tg3-Cre;Ai14-346622.05.02.02

In [19]:
final_df.tail()

recording_date     project                   cell_specimen_name  \
Date                                                                       
180213  2/13/2018 19:05  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.03   
180213  2/13/2018 18:09  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.01.01   
180213  2/13/2018 18:15  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.01   
180213  2/13/2018 18:41  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.02   
180213  2/13/2018 18:45  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.04.01.01   

       ims63x_go_no_go  ephys_seal_gohm       anno.sample_id  \
Date                                                           
180213             NaN         2.400271  PBS4_180213_503_A01   
180213             NaN         2.483585  P9S4_180213_401_A01   
180213             NaN         1.208281  PBS4_180213_501_A01   
180213             NaN         2.977548  PBS4_180213_502_A01   
180213             NaN         4.834966  P1S4_180213_001_A01   

        anno.Total_time_label  anno.Ephys_time_label  \
Date                                                   
180213                    672                     45   
180213                    838                     43   
180213                    921                     46   
180213                    948                     37   
180213                    497                     48   

        anno.Retraction_time_label anno.postPatch_label  \
Date                                                      
180213                           0       nucleus_absent   
180213                         231       nucleus_absent   
180213                           0      nucleus_present   
180213                           0       nucleus_absent   
180213                          78       nucleus_absent   

        anno.endPipetteR_label patched_cell_container  \
Date                                                    
180213                     0.0    PBS4_180213_503_A01   
180213                   900.0    P9S4_180213_401_A01   
180213                     0.0    PBS4_180213_501_A01   
180213                  1000.0    PBS4_180213_502_A01   
180213                  1500.0    P1S4_180213_001_A01   

                                       name  
Date                                         
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.03  
180213  Pvalb-IRES-Cre;Ai14-375845.03.01.01  
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.01  
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.02  
180213  Pvalb-IRES-Cre;Ai14-375845.04.01.01

##### Renaming Duration Columns

In [20]:
#Duration columns are in seconds
final_df.rename(columns = {'anno.Total_time_label' : 'Patch Duration', 
                           'anno.Ephys_time_label' : 'Extraction Duration',
                           'anno.Retraction_time_label' : 'Retraction Duration'}, inplace = True)

In [21]:
final_df.head()

recording_date project                    cell_specimen_name  \
Date                                                                   
170901  9/1/2017 17:46   T301x  Htr3a-Cre_NO152;Ai14-343221.05.01.01   
170901  9/1/2017 17:36   T301x  Htr3a-Cre_NO152;Ai14-343221.05.02.01   
170901  9/1/2017 20:09   T301x   Scnn1a-Tg3-Cre;Ai14-346622.05.02.01   
170901  9/1/2017 20:47   T301x   Scnn1a-Tg3-Cre;Ai14-346622.04.02.01   
170901  9/1/2017 20:31   T301x   Scnn1a-Tg3-Cre;Ai14-346622.05.02.02   

       ims63x_go_no_go  ephys_seal_gohm       anno.sample_id  Patch Duration  \
Date                                                                           
170901          63x go         2.897146  P2S4_170901_051_A01               0   
170901       63x no go         2.461183  P8S4_170901_351_A01               0   
170901       63x no go         3.079292  P8S4_170901_353_A01               0   
170901          63x go         1.452649  P2S4_170901_054_A01               0   
170901       63x no go         2.386576  P8S4_170901_354_A01               0   

        Extraction Duration  Retraction Duration anno.postPatch_label  \
Date                                                                    
170901                    0                    0            Nucleated   
170901                    0                    0              No-Seal   
170901                    0                    0              No-Seal   
170901                    0                    0      Partial-Nucleus   
170901                    0                    0              No-Seal   

        anno.endPipetteR_label patched_cell_container  \
Date                                                    
170901                     0.0    P2S4_170901_051_A01   
170901                     0.0    P8S4_170901_351_A01   
170901                     0.0    P8S4_170901_353_A01   
170901                     0.0    P2S4_170901_054_A01   
170901                     0.0    P8S4_170901_354_A01   

                                        name  
Date                                          
170901  Htr3a-Cre_NO152;Ai14-343221.05.01.01  
170901  Htr3a-Cre_NO152;Ai14-343221.05.02.01  
170901   Scnn1a-Tg3-Cre;Ai14-346622.05.02.01  
170901   Scnn1a-Tg3-Cre;Ai14-346622.04.02.01  
170901   Scnn1a-Tg3-Cre;Ai14-346622.05.02.02

In [22]:
final_df.tail()

recording_date     project                   cell_specimen_name  \
Date                                                                       
180213  2/13/2018 19:05  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.03   
180213  2/13/2018 18:09  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.01.01   
180213  2/13/2018 18:15  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.01   
180213  2/13/2018 18:41  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.02   
180213  2/13/2018 18:45  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.04.01.01   

       ims63x_go_no_go  ephys_seal_gohm       anno.sample_id  Patch Duration  \
Date                                                                           
180213             NaN         2.400271  PBS4_180213_503_A01             672   
180213             NaN         2.483585  P9S4_180213_401_A01             838   
180213             NaN         1.208281  PBS4_180213_501_A01             921   
180213             NaN         2.977548  PBS4_180213_502_A01             948   
180213             NaN         4.834966  P1S4_180213_001_A01             497   

        Extraction Duration  Retraction Duration anno.postPatch_label  \
Date                                                                    
180213                   45                    0       nucleus_absent   
180213                   43                  231       nucleus_absent   
180213                   46                    0      nucleus_present   
180213                   37                    0       nucleus_absent   
180213                   48                   78       nucleus_absent   

        anno.endPipetteR_label patched_cell_container  \
Date                                                    
180213                     0.0    PBS4_180213_503_A01   
180213                   900.0    P9S4_180213_401_A01   
180213                     0.0    PBS4_180213_501_A01   
180213                  1000.0    PBS4_180213_502_A01   
180213                  1500.0    P1S4_180213_001_A01   

                                       name  
Date                                         
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.03  
180213  Pvalb-IRES-Cre;Ai14-375845.03.01.01  
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.01  
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.02  
180213  Pvalb-IRES-Cre;Ai14-375845.04.01.01

In [23]:
#Changed Time from seconds to minutes
final_df['Patch Duration'] = final_df['Patch Duration'] / 60
final_df['Extraction Duration'] = final_df['Extraction Duration'] / 60
final_df['Retraction Duration'] = final_df['Retraction Duration'] / 60

In [24]:
final_df.tail()

recording_date     project                   cell_specimen_name  \
Date                                                                       
180213  2/13/2018 19:05  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.03   
180213  2/13/2018 18:09  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.01.01   
180213  2/13/2018 18:15  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.01   
180213  2/13/2018 18:41  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.03.02.02   
180213  2/13/2018 18:45  mIVSCC-MET  Pvalb-IRES-Cre;Ai14-375845.04.01.01   

       ims63x_go_no_go  ephys_seal_gohm       anno.sample_id  Patch Duration  \
Date                                                                           
180213             NaN         2.400271  PBS4_180213_503_A01       11.200000   
180213             NaN         2.483585  P9S4_180213_401_A01       13.966667   
180213             NaN         1.208281  PBS4_180213_501_A01       15.350000   
180213             NaN         2.977548  PBS4_180213_502_A01       15.800000   
180213             NaN         4.834966  P1S4_180213_001_A01        8.283333   

        Extraction Duration  Retraction Duration anno.postPatch_label  \
Date                                                                    
180213             0.750000                 0.00       nucleus_absent   
180213             0.716667                 3.85       nucleus_absent   
180213             0.766667                 0.00      nucleus_present   
180213             0.616667                 0.00       nucleus_absent   
180213             0.800000                 1.30       nucleus_absent   

        anno.endPipetteR_label patched_cell_container  \
Date                                                    
180213                     0.0    PBS4_180213_503_A01   
180213                   900.0    P9S4_180213_401_A01   
180213                     0.0    PBS4_180213_501_A01   
180213                  1000.0    PBS4_180213_502_A01   
180213                  1500.0    P1S4_180213_001_A01   

                                       name  
Date                                         
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.03  
180213  Pvalb-IRES-Cre;Ai14-375845.03.01.01  
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.01  
180213  Pvalb-IRES-Cre;Ai14-375845.03.02.02  
180213  Pvalb-IRES-Cre;Ai14-375845.04.01.01

In [25]:
#Filtering 
#dropNan's in ims63x_go_no_go